In [3]:
!conda install -c conda-forge folium=0.5.0 --yes


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
import pandas as pd
import folium


In [19]:
# @hidden_cell
CLIENT_ID = '04ZWA5YQNRLNG4MT4MMPWELMAM4G0WVGCGJZA2F42SZ5QM5S'
CLIENT_SECRET = '14VMR3TTDBW2PIR201NDJ0BVK0MCKQ1N0P0DNV0EENZAYPBH'

In [6]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Istanbul",
    "radius": 100000,
    "limit": 10000}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [7]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [8]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Istanbul', 'Istanbul')

In [9]:
d["suggestedBounds"], d["totalResults"]


({'ne': {'lat': 41.08303114156402, 'lng': 29.061183788394654},
  'sw': {'lat': 40.96357998724501, 'lng': 28.847206452111866}},
 249)

In [10]:
d["geocode"]


{'what': '',
 'where': 'istanbul',
 'center': {'lat': 41.01384, 'lng': 28.94966},
 'displayString': 'Istanbul, Turkey',
 'cc': 'TR',
 'geometry': {'bounds': {'ne': {'lat': 41.583896981907635,
    'lng': 29.95880097737308},
   'sw': {'lat': 40.80260400000006, 'lng': 27.971700000085676}}},
 'slug': 'istanbul',
 'longId': '72057594038672980'}

In [11]:
d["groups"][0].keys()


dict_keys(['type', 'name', 'items'])

In [12]:
d["groups"][0]["type"], d["groups"][0]["name"]


('Recommended Places', 'recommended')

In [13]:

items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d9268cd69aaf30008fff5c1',
  'name': 'Dükkan Galata',
  'location': {'address': 'Müeyyedzade Mah. Tatarbeyi Sok. 15/B Galata-Beyoğlu',
   'crossStreet': 'Galata',
   'lat': 41.025881,
   'lng': 28.97539,
   'labeledLatLngs': [{'label': 'display', 'lat': 41.025881, 'lng': 28.97539}],
   'postalCode': '34425',
   'cc': 'TR',
   'neighborhood': 'Müeyyedzade',
   'city': 'İstanbul',
   'state': 'İstanbul',
   'country': 'Türkiye',
   'formattedAddress': ['Müeyyedzade Mah. Tatarbeyi Sok. 15/B Galata-Beyoğlu (Galata)',
    '34425 İstanbul',
    'Türkiye']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups'

In [14]:
items[1]


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ba63359f964a5200b3b39e3',
  'name': 'Vefa Bozacısı',
  'location': {'address': 'Mollahüsrev Mah. Vefa Cad. No:66',
   'lat': 41.01525972294839,
   'lng': 28.958503585514553,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.01525972294839,
     'lng': 28.958503585514553}],
   'postalCode': '34134',
   'cc': 'TR',
   'neighborhood': 'Mollahüsrev',
   'city': 'Fatih',
   'state': 'İstanbul',
   'country': 'Türkiye',
   'formattedAddress': ['Mollahüsrev Mah. Vefa Cad. No:66',
    '34134 Fatih',
    'Türkiye']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-4ba6335

In [15]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue['location']
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    if not "address" in location:
        continue
    address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()


found 48 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,5d9268cd69aaf30008fff5c1,Dükkan Galata,Café,Müeyyedzade Mah. Tatarbeyi Sok. 15/B Galata-Be...,34425,41.025881,28.975390
1,4ba63359f964a5200b3b39e3,Vefa Bozacısı,Café,Mollahüsrev Mah. Vefa Cad. No:66,34134,41.015260,28.958504
2,524a795011d20ce6648c5579,Velvet Cafe,Café,Kule Çıkmazı 7A Galata Beyoğlu,34420,41.025631,28.973714
3,559922f8498e7e8184a87be5,Cumbalı Kahve,Coffee Shop,Ayvansaray mahallesi Kürkçü Çeşmesi Sokak No:1...,34087,41.032992,28.945756
4,59b8dea63149b905ced9edc5,Hamur İşi Cafe & Pastane,Coffee Shop,Haraççıbaşı Sok. No:6A,34083,41.026882,28.955411


In [16]:
istanbul_center = d["geocode"]["center"]
istanbul_center


{'lat': 41.01384, 'lng': 28.94966}

In [17]:
from folium import plugins

# create map of Istanbul using latitude and longitude values
map_istanbul = folium.Map(location=[istanbul_center["lat"], istanbul_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_istanbul)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_istanbul.add_child(plugins.HeatMap(hm_data))

map_istanbul


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [18]:
lat = 40.991099
lng = 29.022015
map_istanbul = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_istanbul)
map_istanbul
